## Handling Missing Values in Large-scale ML Pipelines:

**Task 1**: Impute with Mean or Median
- Step 1: Load a dataset with missing values (e.g., Boston Housing dataset).
- Step 2: Identify columns with missing values.
- Step 3: Impute missing values using the mean or median of the respective columns.

In [ ]:
# write your code from here

**Task 2**: Impute with the Most Frequent Value
- Step 1: Use the Titanic dataset and identify columns with missing values.
- Step 2: Impute categorical columns using the most frequent value.

In [ ]:
# write your code from here

**Task 3**: Advanced Imputation - k-Nearest Neighbors
- Step 1: Implement KNN imputation using the KNNImputer from sklearn.
- Step 2: Explore how KNN imputation improves data completion over simpler methods.

In [ ]:
# write your code from here

## Feature Scaling & Normalization Best Practices:

**Task 1**: Standardization
- Step 1: Standardize features using StandardScaler.
- Step 2: Observe how standardization affects data distribution.

In [ ]:
# write your code from here

**Task 2**: Min-Max Scaling

- Step 1: Scale features to lie between 0 and 1 using MinMaxScaler.
- Step 2: Compare with standardization.

In [ ]:
# write your code from here

**Task 3**: Robust Scaling
- Step 1: Scale features using RobustScaler, which is useful for data with outliers.
- Step 2: Assess changes in data scaling compared to other scaling methods.

In [ ]:
# write your code from here

## Feature Selection Techniques:
### Removing Highly Correlated Features:

**Task 1**: Correlation Matrix
- Step 1: Compute correlation matrix.
- Step 2: Remove highly correlated features (correlation > 0.9).

In [ ]:
# write your code from here

### Using Mutual Information & Variance Thresholds:

**Task 2**: Mutual Information
- Step 1: Compute mutual information between features and target.
- Step 2: Retain features with high mutual information scores.

In [ ]:
# write your code from here

**Task 3**: Variance Threshold
- Step 1: Implement VarianceThreshold to remove features with low variance.
- Step 2: Analyze impact on feature space.

In [ ]:
# write your code from here

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt

# Task 1: Impute with Mean or Median using Boston Housing dataset
boston = load_boston()
X_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
X_boston.iloc[0, 0] = np.nan  # Introduce missing value

# Impute using mean
imputer_mean = SimpleImputer(strategy='mean')
X_boston_mean = pd.DataFrame(imputer_mean.fit_transform(X_boston), columns=X_boston.columns)

# Task 2: Impute with Most Frequent using Titanic dataset
titanic_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic = pd.read_csv(titanic_url)
cat_cols = titanic.select_dtypes(include='object').columns

imputer_freq = SimpleImputer(strategy='most_frequent')
titanic[cat_cols] = imputer_freq.fit_transform(titanic[cat_cols])

# Task 3: Advanced Imputation - k-Nearest Neighbors
knn_imputer = KNNImputer(n_neighbors=3)
X_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(X_boston), columns=X_boston.columns)

# Feature Scaling & Normalization

# Standardization
scaler_std = StandardScaler()
X_std = pd.DataFrame(scaler_std.fit_transform(X_boston_mean), columns=X_boston.columns)

# Min-Max Scaling
scaler_mm = MinMaxScaler()
X_mm = pd.DataFrame(scaler_mm.fit_transform(X_boston_mean), columns=X_boston.columns)

# Robust Scaling
scaler_robust = RobustScaler()
X_robust = pd.DataFrame(scaler_robust.fit_transform(X_boston_mean), columns=X_boston.columns)

# Feature Selection - Correlation
corr_matrix = X_boston_mean.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
X_uncorrelated = X_boston_mean.drop(columns=to_drop)

# Feature Selection - Mutual Information
# Convert Boston target to classification for MI (using median split)
y_boston = (boston.target > np.median(boston.target)).astype(int)
mi_scores = mutual_info_classif(X_boston_mean, y_boston)
mi_selected = X_boston_mean.columns[mi_scores > np.median(mi_scores)]
X_mi = X_boston_mean[mi_selected]

# Feature Selection - Variance Threshold
selector = VarianceThreshold(threshold=0.01)
X_var = selector.fit_transform(X_boston_mean)
X_var_df = pd.DataFrame(X_var, columns=X_boston_mean.columns[selector.get_support(indices=True)])

# Display processed dataframes
print("Imputed with Mean (Boston):")
print(X_boston_mean.head())

print("\nImputed with Most Frequent (Titanic - Categorical Columns):")
print(titanic[cat_cols].head())

print("\nkNN Imputation (Boston):")
print(X_knn_imputed.head())

print("\nStandardized Data (Boston):")
print(X_std.head())

print("\nMin-Max Scaled Data (Boston):")
print(X_mm.head())

print("\nRobust Scaled Data (Boston):")
print(X_robust.head())

print("\nAfter Correlation-based Feature Selection (Boston):")
print(X_uncorrelated.head())

print("\nAfter Mutual Information Feature Selection (Boston):")
print(X_mi.head())

print("\nAfter Variance Threshold Feature Selection (Boston):")
print(X_var_df.head())


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: